In [ ]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sksurv.nonparametric import kaplan_meier_estimator
from lifelines import KaplanMeierFitter, WeibullFitter
from lifelines.fitters import ParametricUnivariateFitter
import autograd.numpy as anp
import scipy.stats as stats
import scipy as sp


%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
def estimate_survival_curve(df: pd.DataFrame):
    df["Membership Expiration Date"] = pd.to_datetime(df["Membership Expiration Date"])
    df.sort_values(by=["Membership Expiration Date"], ascending=True, inplace=True)

    durations = df.groupby(df["Account ID"])[["Account ID"]].count()
    durations["Account ID"].astype(int)
    durations.rename(columns={"Account ID": "Duration"}, inplace=True)

    latest_expiration = df.groupby(df["Account ID"])[["Membership Expiration Date"]].apply(lambda x: x.tail(1))
    latest_expiration.rename(columns={"Membership Expiration Date": "Latest Membership Expiration"}, inplace=True)

    survival_df = pd.merge(left=durations, right=latest_expiration, on="Account ID")

    survival_df.insert(len(survival_df.columns), "Expired", False)

    survival_df.loc[pd.to_datetime(survival_df["Latest Membership Expiration"]) < pd.Timestamp.today(), "Expired"] = True

    return survival_df

In [ ]:
df1 = pd.read_csv("annual_membership_counts_all_time.csv")
df2 = pd.read_csv("monthly_membership_counts_all_time.csv")

df1 = estimate_survival_curve(df1)
df2 = estimate_survival_curve(df2)


In [ ]:
x1, y1, conf_int_1 = kaplan_meier_estimator(df1["Expired"], df1["Duration"], conf_type="log-log")
x2, y2, conf_int_2 = kaplan_meier_estimator(df2["Expired"], df2["Duration"], conf_type="log-log")

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 6.75), layout="constrained")

axs[0].step(x1, y1, where="post")
axs[0].fill_between(x1, conf_int_1[0], conf_int_1[1], alpha=0.25, step="post")
axs[0].set_ylim(0, 1)
axs[0].set_xlabel("Membership Duration (Years)")
axs[0].set_title("Annual Memberships")
fig.supylabel("Probability of Continued Membership")


axs[1].step(x2, y2, where="post")
axs[1].fill_between(x2, conf_int_2[0], conf_int_2[1], alpha=0.25, step="post")
axs[1].set_ylim(0, 1)
axs[1].set_xlabel("Membership Duration (Months)")
axs[1].set_title("Monthly Memberships")


plt.show()


The survival function is defined as $S(x) = 1 - F(x) = e^{-H(x)}$ where $F(x)$ is the cumulative distribution fucntion and $H(x)$ is the cumulative hazard function.

Thus, the cumulative hazard function can be written as $H(x) = -\ln(1-F(x))$

The cumulative distribution function for the Lomax distribution is $F(x)=1-(1+\frac{x}{\lambda})^{-\kappa}$, thus the cumulative hazard function for the Lomax distribution
is $H(x)= \kappa \ln(1+\frac{x}{\lambda})$

We will use this cumulative hazard function to create a custom parametric fitter using the Lifelines package. We will fit the inverse of the constant $\lambda$ to avoid negative values in the diagonal of the covariance matrix. Later, we'll simply invert the fitted constant to find the true $\lambda$.

In [ ]:
class LomaxFitter(ParametricUnivariateFitter):

    _fitted_parameter_names = ["kappa_", "inverse_lambda"]

    def _cumulative_hazard(self, params, times):
        kappa, inverse_lambda = params
        return kappa * anp.log(1 + times*inverse_lambda)

In [ ]:
kmf = KaplanMeierFitter()
wbf = WeibullFitter()
lmf = LomaxFitter()

In [ ]:
ax = plt.subplot(111)

kmf.fit(df1["Duration"]*12, event_observed=df1["Expired"], label="KM Estimate (Annual)")
lmf.fit(df1["Duration"]*12, event_observed=df1["Expired"], label="Lomax Fit (Annual)")
kmf.plot_survival_function(ax=ax)
lmf.plot_survival_function(ax=ax)

kmf.fit(durations=df2["Duration"], event_observed=df2["Expired"], label="KM Estimate (Monthly)")
lmf.fit(durations=df2["Duration"], event_observed=df2["Expired"], label="Lomax Fit (Monthly)")
kmf.plot_survival_function(ax=ax)
lmf.plot_survival_function(ax=ax)

ax.set_xlabel("Membership Duration (Months)")
ax.set_ylabel("Probability of Continued Membership")

plt.savefig("membership_kaplan_meier_estimates_combined", dpi=300)

The mean of the Lomax distribution is defined as $E[X]=\frac{\lambda}{\kappa-1}\;\text{for}\; \kappa>1,\; \text{else undefined}$, which in this case represents the expected duration of membership. Customer lifetime value is then simply the expected duration of membership multiplied by the average revenue per customer per unit of duration, in this case average revenue per month per subscriber.

The median of the Lomax distribution is defined as $\lambda(2^{1/\kappa}-1)$, which represents the duration at which there is a 50% probability that a member will still be subscribed. Stated another way, it is the duration at which we can expect 50% of new members to have churned.

In [ ]:
lambda_ = 1/lmf.inverse_lambda
kappa = lmf.kappa_

lomax_mean = np.divide(lambda_, (kappa - 1))

print("lambda:", lambda_, "kappa:", kappa)
print("Lomax Median Survival for Monthly Memberships:", np.round(lmf.median_survival_time_, 2), "Months")
print("Lomax Mean Survival for Monthly Memberships:", np.round(lomax_mean, 2), "Months")
print("Kaplan-Meier Median Survival for Monthly Memberships:", kmf.median_survival_time_, "Months")

In [ ]:
lmf.fit(df1["Duration"]*12, event_observed=df1["Expired"])
kmf.fit(df1["Duration"]*12, event_observed=df1["Expired"])

lambda_ = 1/lmf.inverse_lambda
kappa = lmf.kappa_

print("lambda:", lambda_, "kappa:", kappa)
print("Lomax Median Survival for Annual Memberships:", np.round(lmf.median_survival_time_, 2), "Months")
print("Lomax Mean Survival for Annual Memberships:", np.round((lambda_ / (kappa - 1)), 2), "Months")
print("Kaplan-Meier Median Survival for Annual Memberships:", kmf.median_survival_time_, "Months")

So we see that our final estimates from fitting the Lomax distribution are:
        
|          | Monthly Membership | Annual Membership  |
|     ----  | :----:      | :----: |
| Median Duration (Months)    | 7.66    | 45.61   |
| Mean Duration (Months)      | 21.87    | 68.59  |

In [ ]:
censored_data = df2.loc[~df2["Expired"], "Duration"]
uncensored_data = df2.loc[df2["Expired"] == True, "Duration"]

In [ ]:
def log_likelihood_lomax(args):
    shape, scale = args
    val = stats.lomax.logpdf(uncensored_data, shape, loc=0, scale=scale).sum() + stats.lomax.logsf(censored_data, shape, loc=0, scale=scale).sum()
    return -val

In [ ]:
res_lomax = sp.optimize.minimize(log_likelihood_lomax, [1, 1], bounds=((0.001, 1000000), (0.001, 1000000)))

print("lomax shape", res_lomax.x[0], ", scale=", res_lomax.x[1])
print("lomax mean", stats.lomax.mean(res_lomax.x[0], scale=res_lomax.x[1]))
print("lomax median", stats.lomax.median(res_lomax.x[0], scale=res_lomax.x[1]))